<a href="https://colab.research.google.com/github/PrinceVerma04/FINMEE-Report/blob/main/FINMEE_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
rehandle_df = pd.read_csv("/content/rehandle_logs.csv")
truck_df = pd.read_csv("/content/truck_dump_logs.csv")


In [4]:
rehandle_df.head()



,id,shift_date,material,loading_location,dumping_location,nominal_tons,loading_eqmt,time_loading,dump_kind
0,50000,06/18/2025,High Grade,Finger 1,Main Crusher,23.5,992K,06/18/2025 13:39,Correct Dump
1,50001,06/23/2025,High Grade,Finger 1,Main Crusher,20.3,992K,06/23/2025 20:47,Correct Dump
2,50002,07/08/2025,High Grade,Finger 1,Main Crusher,22.2,992K,07/08/2025 08:35,Correct Dump
3,50003,07/05/2025,High Grade,Finger 1,Main Crusher,22.6,992K,07/05/2025 12:20,Correct Dump
4,50004,06/19/2025,High Grade,Finger 1,Main Crusher,21.6,992K,06/19/2025 06:52,Correct Dump


In [6]:
rehandle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                50000 non-null  int64  
 1   shift_date        50000 non-null  object 
 2   material          50000 non-null  object 
 3   loading_location  50000 non-null  object 
 4   dumping_location  50000 non-null  object 
 5   nominal_tons      50000 non-null  float64
 6   loading_eqmt      50000 non-null  object 
 7   time_loading      50000 non-null  object 
 8   dump_kind         50000 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.4+ MB


In [8]:
rehandle_df.shape

(50000, 9)

In [5]:
truck_df.head()

,id,shift_date,material,loading_location,dumping_location,nominal_tons,hauling_eqmt,time_dumping,dump_kind
0,1000,2025-06-21,Low Grade,P2-Pit,Finger 2,237.0,777G,2025-06-21 09:54:00,Correct Dump
1,1001,2025-07-10,High Grade,P1-Pit,Finger 1,228.0,777G,2025-07-10 15:14:00,Correct Dump
2,1002,2025-06-15,Low Grade,P2-Pit,Finger 2,224.8,777G,15/06/2025 05:26 PM (NZDT),Correct Dump
3,1003,2025-06-21,Low Grade,P2-Pit,Finger 2,235.4,777G,2025-06-21 19:24:00,Correct Dump
4,1004,2025-07-01,Low Grade,P2-Pit,Finger 2,235.9,777G,2025-07-01 15:47:00,Correct Dump


In [9]:
truck_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                50000 non-null  int64  
 1   shift_date        50000 non-null  object 
 2   material          50000 non-null  object 
 3   loading_location  50000 non-null  object 
 4   dumping_location  50000 non-null  object 
 5   nominal_tons      50000 non-null  float64
 6   hauling_eqmt      50000 non-null  object 
 7   time_dumping      50000 non-null  object 
 8   dump_kind         50000 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.4+ MB


In [10]:
truck_df.shape

(50000, 9)

## material Information flow
P1-Pit (High Grade)
   └── Truck (777G)
        └── Finger 1 (Stockpile)
             └── Loader (992K)
                  └── Main Crusher

P2-Pit (Low Grade)
   └── Truck (777G)
        └── Finger 2 (Stockpile)
             └── Loader (992K)
                  └── Main Crusher


DATA CLEANING + STANDARDIZATION

Rehandle logs

In [11]:
rehandle_df["shift_date"] = pd.to_datetime(
    rehandle_df["shift_date"], errors="coerce"
)

rehandle_df["time_loading"] = pd.to_datetime(
    rehandle_df["time_loading"], errors="coerce"
)


Truck dump logs

mixed format

In [12]:
truck_df["shift_date"] = pd.to_datetime(
    truck_df["shift_date"], errors="coerce"
)

truck_df["time_dumping"] = pd.to_datetime(
    truck_df["time_dumping"], errors="coerce"
)


In [14]:
rehandle_df.dtypes



,0
id,int64
shift_date,datetime64[ns]
material,object
loading_location,object
dumping_location,object
nominal_tons,float64
loading_eqmt,object
time_loading,datetime64[ns]
dump_kind,object


In [15]:
truck_df.dtypes

,0
id,int64
shift_date,datetime64[ns]
material,object
loading_location,object
dumping_location,object
nominal_tons,float64
hauling_eqmt,object
time_dumping,datetime64[ns]
dump_kind,object


In [16]:
rehandle_df = rehandle_df[rehandle_df["dump_kind"] == "Correct Dump"]
truck_df = truck_df[truck_df["dump_kind"] == "Correct Dump"]


Material Flow Sanity Check

We check here HG finger should not go in wrong finger

In [19]:
truck_df.groupby(
    ["loading_location", "dumping_location", "material"]
).size().reset_index(name="count")


,loading_location,dumping_location,material,count
0,P1-Pit,Finger 1,High Grade,27767
1,P1-Pit,Finger 2,High Grade,876
2,P2-Pit,Finger 1,Low Grade,538
3,P2-Pit,Finger 2,Low Grade,18344


DAILY VOLUME AGGREGATION

Truck: Pit → Finger (Daily)

In [20]:
truck_daily = (
    truck_df
    .groupby(["shift_date", "dumping_location", "material"])
    .agg(
        pit_tons=("nominal_tons", "sum"),
        truck_trips=("id", "count")
    )
    .reset_index()
)

truck_daily.head()


,shift_date,dumping_location,material,pit_tons,truck_trips
0,2025-06-15,Finger 1,High Grade,167746.9,730
1,2025-06-15,Finger 1,Low Grade,2049.2,9
2,2025-06-15,Finger 2,High Grade,4393.5,19
3,2025-06-15,Finger 2,Low Grade,109025.7,473
4,2025-06-16,Finger 1,High Grade,235313.1,1022


Rehandle: Finger → Crusher (Daily)

In [21]:
rehandle_daily = (
    rehandle_df
    .groupby(["shift_date", "loading_location", "material"])
    .agg(
        crusher_tons=("nominal_tons", "sum"),
        loader_cycles=("id", "count")
    )
    .reset_index()
)

rehandle_daily.head()


,shift_date,loading_location,material,crusher_tons,loader_cycles
0,2025-06-15,Finger 1,High Grade,14905.4,662
1,2025-06-15,Finger 2,Low Grade,15603.4,692
2,2025-06-16,Finger 1,High Grade,20254.4,900
3,2025-06-16,Finger 2,Low Grade,21299.2,945
4,2025-06-17,Finger 1,High Grade,20473.4,913


In [22]:
rehandle_daily.rename(
    columns={"loading_location": "dumping_location"},
    inplace=True
)


DAILY VOLUME MATCH

In [23]:
daily_balance = truck_daily.merge(
    rehandle_daily,
    on=["shift_date", "dumping_location", "material"],
    how="left"
)

daily_balance["recovery_pct"] = (
    daily_balance["crusher_tons"] /
    daily_balance["pit_tons"]
) * 100

daily_balance.head()


,shift_date,dumping_location,material,pit_tons,truck_trips,crusher_tons,loader_cycles,recovery_pct
0,2025-06-15,Finger 1,High Grade,167746.9,730,14905.4,662.0,8.885649
1,2025-06-15,Finger 1,Low Grade,2049.2,9,NaN,NaN,NaN
2,2025-06-15,Finger 2,High Grade,4393.5,19,NaN,NaN,NaN
3,2025-06-15,Finger 2,Low Grade,109025.7,473,15603.4,692.0,14.311671
4,2025-06-16,Finger 1,High Grade,235313.1,1022,20254.4,900.0,8.607426


“More than 90% of the material is not reaching the crusher on the same day.”

In [24]:
daily_balance.sort_values("recovery_pct").head(10)


,shift_date,dumping_location,material,pit_tons,truck_trips,crusher_tons,loader_cycles,recovery_pct
48,2025-06-27,Finger 1,High Grade,238945.7,1040,19599.3,873.0,8.202407
20,2025-06-20,Finger 1,High Grade,253329.6,1101,20841.8,926.0,8.227148
84,2025-07-06,Finger 1,High Grade,241154.5,1048,20310.5,904.0,8.422194
52,2025-06-28,Finger 1,High Grade,237703.4,1034,20049.7,892.0,8.434755
108,2025-07-12,Finger 1,High Grade,237081.8,1032,20142.6,896.0,8.496055
60,2025-06-30,Finger 1,High Grade,237379.6,1031,20345.5,904.0,8.570871
104,2025-07-11,Finger 1,High Grade,233821.7,1017,20052.4,891.0,8.575936
72,2025-07-03,Finger 1,High Grade,227597.6,991,19520.2,867.0,8.576628
4,2025-06-16,Finger 1,High Grade,235313.1,1022,20254.4,900.0,8.607426
32,2025-06-23,Finger 1,High Grade,242123.9,1053,20928.5,933.0,8.643715


volume mismatch” fear 100%

Same Finger → Multiple Materials?

In [25]:
rehandle_df.groupby(
    ["loading_location", "shift_date"]
)["material"].nunique().reset_index(name="material_count")


,loading_location,shift_date,material_count
0,Finger 1,2025-06-15,1
1,Finger 1,2025-06-16,1
2,Finger 1,2025-06-17,1
3,Finger 1,2025-06-18,1
4,Finger 1,2025-06-19,1
5,Finger 1,2025-06-20,1
6,Finger 1,2025-06-21,1
7,Finger 1,2025-06-22,1
8,Finger 1,2025-06-23,1
9,Finger 1,2025-06-24,1


on same day loader load same type of material

## Important nuance:

Dilution is not happening through same-day loader operations.
Dilution is occurring at the truck dumping stage.


In [28]:
mixed_days = rehandle_df.groupby(
    ["loading_location", "shift_date"]
)["material"].nunique().reset_index()

mixed_days[mixed_days["material"] > 1]


,loading_location,shift_date,material


KEY RESULTS:
Evidence #1 (Routing Issue)

HG → Finger 2

LG → Finger 1

Evidence #2 (Massive Volume Mismatch)

HG Recovery ≈ 8–9%

LG Recovery ≈ 14%

Evidence #3 (Root Cause Isolation)

Rehandle process clean

Issue upstream (truck dumping + stockpile logic)

# Where is dilution happening? (Root Cause Analysis)

mixed_days[mixed_days["material"] > 1]


Result:
Empty DataFrame

Interpretation:

On the same finger and same day, only one material is present

Loader is not mixing materials

Actual Root Cause Identified:

Truck dumping stage

Incorrect routing at stockpile entry

Trucks are dumping material in the wrong stockpiles, leading to dilution

“Lead Pitch”: What will we show on the dashboard?

 Single Most Important Metric:
 Daily Ore Recovery % (Pit → Crusher)

By Material

By Finger

Alert if < 90%

At a single glance, the manager should be able to say:

“The problem is right here.”